In [39]:
# A decision tree is a flowchart-like structure where each internal node represents a feature (or attribute),
# each branch represents a decision rule, and each leaf node represents the outcome.
# It effectively divides the data into subsets based on different features to make predictions or decisions.

In [68]:
#The goal is to create a tree that minimizes impurity or maximizes information gain at each node.

In [81]:
import numpy as np
from sklearn.metrics import accuracy_score

In [78]:
class DecisionTree:
    def __init__(self):
        self.tree = {}

# Entropy to calculate the Impurity
# I(Dp,D_left,D_right)=-Sum(p)log2(p)
    def entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return -np.sum(probs * np.log2(probs))

# Split data for every feature:
# If X=[[6, 1], [3, 1], [4, 0], [2, 1], [1, 0], [5, 0]]
# n_features=2
    def find_best_split(self, X, y):
        best_split = {}
        best_info_gain = -1
        n_features = X.shape[1]

        for feature in range(n_features):
            feature_values = np.unique(X[:, feature])# feature_values=0,1
            for value in feature_values:
                left_indices = np.where(X[:, feature] <= value)[0]
                right_indices = np.where(X[:, feature] > value)[0]

                left_labels = y[left_indices]#D(left)
                right_labels = y[right_indices]#D(right)

                info_gain = self.info_gain(y, left_labels, right_labels)
                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_split = {
                        'feature_index': feature,
                        'threshold': value,
                        'left_indices': left_indices,
                        'right_indices': right_indices
                    }

        return best_split

# IG(Dp,F)=I(Dp)-N_left/Np*I(D_left)-N_right/Np*I(D_right).
    def info_gain(self, y, left_labels, right_labels):
        p = len(left_labels) / len(y)
        q = len(right_labels) / len(y)
        return self.entropy(y) - (p * self.entropy(left_labels) + q * self.entropy(right_labels))

    
# stopping criterion:
# 1)Reaching minimum number of samples : 
    def build_tree(self, X, y, depth=0, max_depth=5):
        if len(np.unique(y)) == 1:
            major_class = np.unique(y)[0]
            return major_class

# 2)Reaching a maximum depth :
        if depth == max_depth:
            major_class = np.unique(y)[np.argmax(np.bincount(y))]
            return major_class

        best_split = self.find_best_split(X, y)
        feature_index = best_split['feature_index']
        threshold = best_split['threshold']
        left_indices = best_split['left_indices']
        right_indices = best_split['right_indices']

        left_node = self.build_tree(X[left_indices], y[left_indices], depth + 1, max_depth)
        right_node = self.build_tree(X[right_indices], y[right_indices], depth + 1, max_depth)

        self.tree[(feature_index, threshold)] = {'left': left_node, 'right': right_node}

    def predict(self, X):
        predictions = []
        for sample in X:
            node = self.tree
            while isinstance(node, dict):
                feature_index, threshold = list(node.keys())[0]
                if sample[feature_index] <= threshold:
                    node = node[(feature_index, threshold)]['left']
                else:
                    node = node[(feature_index, threshold)]['right']
            predictions.append(node)
        return np.array(predictions)

In [79]:
X = np.array([[6, 1], [3, 1], [4, 0], [2, 1], [1, 0], [5, 0]])
y = np.array([1, 0, 0, 1, 0, 1])

tree = DecisionTree()
tree.build_tree(X, y)

X_test = np.array([[2, 1], [6, 0]])
predictions = tree.predict(X_test)

print(predictions)

[1 1]


In [80]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

clf.fit(X, y)


clf.predict(X_test)

array([1, 1])

## Done...